In [ ]:
import collections

import cvxpy as cp
import lightgbm as lgb
import numpy as np
import pandas as pd
from prophet import Prophet
from statsmodels.tsa.arima.model import ARIMA
import xgboost as xgb

from hierarchical_forecast.base_forecast import *
from hierarchical_forecast.previous import *
from hierarchical_forecast.proposal import *
from hierarchical_forecast.evaluation import *

# Data

In [2]:
# Australian birth data

# import data
df_bottom = pd.read_csv("data/1_birth.csv", index_col=0)
df_bottom.index = pd.to_datetime(df_bottom.index)

# define hierarchy
agg_list = ["total"]
df_agg = []
columns_agg = []
df_agg += [df_bottom.sum(axis=1)]
columns_agg += [df_bottom.columns]
df_agg = pd.concat(df_agg, axis=1, keys=agg_list)

# generate observation DataFrame
df = pd.concat([df_agg, df_bottom], axis=1)

# split data
df_train = df[df.index < "2018-01-01"]
df_test = df[df.index >= "2018-01-01"]

# extract parameters
num_train = df_train.shape[0]
num_test = df_test.shape[0]
num_bottom = df_bottom.shape[1]
num_agg = df_agg.shape[1]

In [2]:
# Australian tourism data

# import data
df_bottom = pd.read_csv("data/2_tourism.csv", index_col=0)
df_bottom.index = pd.to_datetime(df_bottom.index)

# define hierarchy
agg_list = ["total", "NSW", "NT", "QLD", "SA", "TAS", "VIC", "WA"]
df_agg = []
columns_agg = []
df_agg += [df_bottom.sum(axis=1)]
columns_agg += [df_bottom.columns]
for column in agg_list[1:]:
    df_agg_i = df_bottom.filter(like=column)
    df_agg += [df_agg_i.sum(axis=1)]
    columns_agg += [df_agg_i.columns]
df_agg = pd.concat(df_agg, axis=1, keys=agg_list)

# generate observation DataFrame
df = pd.concat([df_agg, df_bottom], axis=1)

# split data
df_train = df[df.index < "2015-01-01"]
df_test = df[df.index >= "2015-01-01"]

# extract parameters
num_train = df_train.shape[0]
num_test = df_test.shape[0]
num_bottom = df_bottom.shape[1]
num_agg = df_agg.shape[1]

In [33]:
# Walmart sales data

# import data
df_bottom = pd.read_csv("data/3_walmart-sales.csv", index_col=0)
df_bottom.index = pd.to_datetime(df_bottom.index)

# define hierarchy
agg_list = ["total", "CA", "TX", "WI"]
df_agg = []
columns_agg = []
df_agg += [df_bottom.sum(axis=1)]
columns_agg += [df_bottom.columns]
for column in agg_list[1:]:
    df_agg_i = df_bottom.filter(like=column)
    df_agg += [df_agg_i.sum(axis=1)]
    columns_agg += [df_agg_i.columns]
df_agg = pd.concat(df_agg, axis=1, keys=agg_list)

# generate observation DataFrame
df = pd.concat([df_agg, df_bottom], axis=1)

# split data
df_train = df[df.index < "2016-01-04"]
df_test = df[df.index >= "2016-01-04"]

# extract parameters
num_train = df_train.shape[0]
num_test = df_test.shape[0]
num_bottom = df_bottom.shape[1]
num_agg = df_agg.shape[1]

In [46]:
# Swiss electricity demnad data

# import data
df_bottom = pd.read_csv("data/4_electricity-demand.csv", index_col=0)
df_bottom.index = pd.to_datetime(df_bottom.index)

# define hierarchy
agg_list = ["grid", "S1", "S2", "S11", "S12", "S21", "S22"]
df_agg = []
columns_agg = []
df_agg += [df_bottom.sum(axis=1)]
columns_agg += [df_bottom.columns]
for column in agg_list[1:]:
    df_agg_i = df_bottom.filter(like=column)
    df_agg += [df_agg_i.sum(axis=1)]
    columns_agg += [df_agg_i.columns]
df_agg = pd.concat(df_agg, axis=1, keys=agg_list)

# generate observation DataFrame
df = pd.concat([df_agg, df_bottom], axis=1)

# split data
df_train = df[df.index < "2019-01-01"]
df_test = df[df.index >= "2019-01-01"]

# extract parameters
num_train = df_train.shape[0]
num_test = df_test.shape[0]
num_bottom = df_bottom.shape[1]
num_agg = df_agg.shape[1]

# Forecast

In [ ]:
# forecasting

# base forecast
df_base = base_prophet(df, num_train)
# df_base = base_arima(df, num_train)
# df_base = base_xgb(df, num_train)
# df_base = base_lgb(df, num_train)

# preivious method
df_bu = bottom_up(df, df_base, num_bottom, num_agg, columns_agg)
df_td = top_down(df, df_base, num_bottom, num_agg, columns_agg)
df_gls = gls(df, df_base, num_bottom, num_agg, columns_agg)
df_mint = mint(df, df_base, num_train, num_bottom, num_agg, columns_agg)

# proposal method
df_robust = robust(
    df,
    df_base,
    num_train,
    num_bottom,
    num_agg,
    columns_agg,
    num_bootstrap=5000,
    alpha_grid=np.arange(0.5, 1.01, 0.1),  # sequence of alpha to be explored
    val_ratio=0.9,  # percentage of the train part devoted to validation
)

# Evaluation

In [ ]:
# evaluation summary

mae, rmse = evaluation(
    df,
    df_base,
    df_bu,
    df_td,
    df_gls,
    df_mint,
    df_robust,
    num_test,
)

In [ ]:
# MAE for each time series

mae

In [ ]:
# relative MAE for each time series

base_col = mae.columns[0]
mae_p = mae.div(mae[base_col], axis=0)
mae_p

In [ ]:
# RMSE for each time series

rmse

In [ ]:
# relative RMSE for each time series

base_col = rmse.columns[0]
rmse_p = rmse.div(rmse[base_col], axis=0)
rmse_p